# Simple pendulum

In [ ]:
import sympy, scipy, matplotlib

from mechanics import *

Definition of the system:

In [ ]:
g, l = symbols('g l',positive=True)

# Generalized coordinates
Q = [alpha] = dynsyms(['alpha'])

# number of particles
n = 1
# spatial dimension
dim = 2

# General system of n particles
P,V = posvel(n,dim)
M   = vector(n,'m')
T   = M.dot( mul(V,V) * ones(dim,1) ) / 2

# Potential energy of constant gravity (sum mgy)
V   = g * M.dot(P[:,-1])

# Cartesian coordinates in terms of generalized coordinates
Gen = mkGen(P, [l*sin(alpha),
                -l*cos(alpha)] )


sys = Dynamics(T, V, Gen, Q, {})

Generalized coordinates:

In [ ]:
disp(sys.Q, 'Q')

Cartesian coordinates:

In [ ]:
[ Eq(u,v) for u,v in Gen.items() ]

Potential energy:

In [ ]:
disp3(Symbol('V'),V,sys.V)

Kinetic energy:

In [ ]:
disp3(Symbol('T'),T,sys.T.simplify())

Lagrangian:

In [ ]:
disp(sys.L.simplify(),'L')

Lagrange equations:

In [ ]:
[ Eq(v,solve(e,v)[0]) for e,v in zip(sys.ecsL,sys.D2) ]

Numerical solution for a particular case:

In [ ]:
case = {g:10, l:1, **val(M,[2])}
sys = Dynamics(T, V, Gen, Q, case)
fps = 30

In [ ]:
sol = nsolve(sys.dotL, T=10, dt=1/fps, q0=[1.5, 0])

graph(sol,sys)

A simple animation:

In [ ]:
def prepare():
    fig = plt.figure(figsize=(4,4))
    ax = plt.gca()
    plt.tight_layout()
    ax.axis('equal')
    plt.tick_params(colors='gray')
    #ax.axis('off')
    plt.close();

    ax.plot([-1,1],[-1,0.1],color='white')

    line1, = ax.plot([],[],lw=2,color='gray')
    line0, = ax.plot([], [], '.-', markersize=30,color='black')

    def draw(k, t, x1,y1):
        line0.set_data([x1],[y1])
        line1.set_data([0,x1],[0,y1])

    return fig, draw

repeat=np.argmin(np.linalg.norm(sol[0,1:]-sol[20:,1:],axis=1))+20

ani = mkAnim(sol, sys, prepare, fps, frames=len(sol[:repeat]))
HTML(ani.to_jshtml())

Generalized moments:

In [ ]:
sys = Dynamics(T, V, Gen, Q, {}, alsoH=True)

In [ ]:
sys.Pec

Hamiltonian:

In [ ]:
disp3(Symbol('H'),sys.H0.simplify(),sys.H.simplify())

T+V

In [ ]:
(sys.T+sys.V).simplify()

Hamilton equations (obtained with the Poisson bracket):

In [ ]:
sys.ecsH

In [ ]:
-sys.H.diff(Q[0])

In [ ]:
sys.H.diff(sys.P[0])

Legendre Transform:

In [ ]:
sys.H.diff(t).simplify()

$$ dH = \underbrace{glm \sin(q)}_{-\dot p}\; dq + \underbrace{\frac{p}{l^2 m}}_{\dot q}\; d p  $$

Numerical solution for a particular case:

In [ ]:
sys = Dynamics(T, V, Gen, Q, {g:10,l:1/2,M[0]:2}, alsoH=True)

In [ ]:
sol=nsolve(sys.dotH,10,1/30,[1.5,0])

graph(sol,sys, moments=True)

In [ ]:
sys = Dynamics(T, V, Gen, Q, {M[0]:1, g:1, l:1}, alsoH=True)
Ham = lambdify(Q+sys.P, sys.H, 'math')

In [ ]:
def phase_space(qs,ps, **args):
    vdot = np.array([[sys.dotH([q,p],0) for q in qs] for p in ps])
    vH   = np.array([[Ham(q,p) for q in qs] for p in np.flipud(ps)])
    plt.imshow(-vH,'gray',extent=[min(qs),max(qs),min(ps),max(ps)]);
    plt.quiver(qs, ps, vdot[:,:,0], vdot[:,:,1], scale_units='xy', **args);
    plt.tick_params(labelsize=8)
    plt.xlabel(f'${vlatex(Q[0])}$'); plt.ylabel(f'${vlatex(sys.P[0])}$');
    plt.title('$H$');

a = np.linspace(-0.2,0.2,15)
p = np.linspace(-0.2,0.2,15)
plt.figure(figsize=(4,4))
phase_space(a,p,color='Red', width=0.002, scale=5)

In [ ]:
a = np.linspace(-1,5,100)
p = np.linspace(-1,1,20)
plt.figure(figsize=(10,5))
phase_space(a,p,color='red',width=0.001,scale=10)